In [86]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import json

In [30]:
def read_links(page_num, model):
    page = f'https://auto.ru/moskva/cars/{model}/used/?year_to=2020&page={page_num}&output_type=table'
    response = requests.get(page)
    soup = BeautifulSoup(response.content.decode("utf-8"), 'html.parser')
    class_ = "Link ListingItemTitle__link"
    page_link = soup.find_all('a', class_=class_)
    hrefs=[]
    for a in page_link:
        hrefs.append(a.get('href', None))
    return hrefs


In [41]:
def links_count(model):
    page = f'https://auto.ru/moskva/cars/{model}/used/?year_to=2020&output_type=table'
    response = requests.get(page)
    soup = BeautifulSoup(response.content.decode("utf-8"), 'html.parser')
    class_ = "Button Button_color_whiteHoverBlue Button_size_s Button_type_link Button_width_default ListingPagination-module__page"
    page_links = soup.find_all('a', class_=class_)
    return int(page_links[-1].find('span', class_="Button__text").text)


In [42]:
links_count('skoda')

62

In [43]:
models = ['skoda', 'audi', 'honda', 'volvo', 'bmw', 'nissan', 'infiniti',
          'mercedes', 'toyota', 'lexus', 'volkswagen', 'mitsubishi']
models_dict = dict()
for model in models:
    models_dict[model] = links_count(model)
models_dict

{'skoda': 62,
 'audi': 74,
 'honda': 26,
 'volvo': 33,
 'bmw': 99,
 'nissan': 89,
 'infiniti': 16,
 'mercedes': 99,
 'toyota': 97,
 'lexus': 29,
 'volkswagen': 99,
 'mitsubishi': 62}

In [50]:
n = 0
for model in models_dict.keys():
    with open(f'links_{model}.csv', 'w') as output:
        print(f'Pages for {model} count = {models_dict[model]}')
        for i in range(models_dict[model]):
            print('Page =', i + 1, 'model =', model)
            hrefs = read_links(i + 1, model)
            for href in hrefs:
                output.write(href)
                output.write('\n')

Pages for bmw count = 99
Page = 1 model = bmw
Page = 2 model = bmw
Page = 3 model = bmw
Page = 4 model = bmw
Page = 5 model = bmw
Page = 6 model = bmw
Page = 7 model = bmw
Page = 8 model = bmw
Page = 9 model = bmw
Page = 10 model = bmw
Page = 11 model = bmw
Page = 12 model = bmw
Page = 13 model = bmw
Page = 14 model = bmw
Page = 15 model = bmw
Page = 16 model = bmw
Page = 17 model = bmw
Page = 18 model = bmw
Page = 19 model = bmw
Page = 20 model = bmw
Page = 21 model = bmw
Page = 22 model = bmw
Page = 23 model = bmw
Page = 24 model = bmw
Page = 25 model = bmw
Page = 26 model = bmw
Page = 27 model = bmw
Page = 28 model = bmw
Page = 29 model = bmw
Page = 30 model = bmw
Page = 31 model = bmw
Page = 32 model = bmw
Page = 33 model = bmw
Page = 34 model = bmw
Page = 35 model = bmw
Page = 36 model = bmw
Page = 37 model = bmw
Page = 38 model = bmw
Page = 39 model = bmw
Page = 40 model = bmw
Page = 41 model = bmw
Page = 42 model = bmw
Page = 43 model = bmw
Page = 44 model = bmw
Page = 45 model 

Page = 25 model = toyota
Page = 26 model = toyota
Page = 27 model = toyota
Page = 28 model = toyota
Page = 29 model = toyota
Page = 30 model = toyota
Page = 31 model = toyota
Page = 32 model = toyota
Page = 33 model = toyota
Page = 34 model = toyota
Page = 35 model = toyota
Page = 36 model = toyota
Page = 37 model = toyota
Page = 38 model = toyota
Page = 39 model = toyota
Page = 40 model = toyota
Page = 41 model = toyota
Page = 42 model = toyota
Page = 43 model = toyota
Page = 44 model = toyota
Page = 45 model = toyota
Page = 46 model = toyota
Page = 47 model = toyota
Page = 48 model = toyota
Page = 49 model = toyota
Page = 50 model = toyota
Page = 51 model = toyota
Page = 52 model = toyota
Page = 53 model = toyota
Page = 54 model = toyota
Page = 55 model = toyota
Page = 56 model = toyota
Page = 57 model = toyota
Page = 58 model = toyota
Page = 59 model = toyota
Page = 60 model = toyota
Page = 61 model = toyota
Page = 62 model = toyota
Page = 63 model = toyota
Page = 64 model = toyota


In [167]:
if os.path.exists("auto.ru"):
    os.remove("auto.ru/*")
else:
    os.mkdir("auto.ru")

In [1]:
#!wget -i auto.ru/links_audi.csv -P auto.ru/audi
#!wget -i auto.ru/links_bmw.csv -P auto.ru/bmw
#!wget -i auto.ru/links_honda.csv -P auto.ru/honda
#!wget -i auto.ru/links_infiniti.csv -P auto.ru/infiniti
#!wget -i auto.ru/links_lexus.csv -P auto.ru/lexus
#!wget -i auto.ru/links_mercedes.csv -P auto.ru/mercedes
#!wget -i auto.ru/links_mitsubishi.csv -P auto.ru/mitsubishi
#!wget -i auto.ru/links_nissan.csv -P auto.ru/nissan
#!wget -i auto.ru/links_skoda.csv -P auto.ru/skoda
#!wget -i auto.ru/links_toyota.csv -P auto.ru/toyota
#!wget -i auto.ru/links_volkswagen.csv -P auto.ru/volkswagen
#!wget -i auto.ru/links_volvo.csv -P auto.ru/volvo

In [246]:
def parse_row(parent, class_):
    row = parent.find('li', class_=class_)
    if row is not None:
        cell = row.find_all('span', class_="CardInfoRow__cell")
        if cell is not None and len(cell) > 1:
            return cell[1].text.replace('\xa0', '')
    return None    

def parse_page(filename):
    car = dict()
    with open(filename, 'r') as page:
        soup = BeautifulSoup(page.read(), 'html.parser')
        head = soup.find('head')
        canonical = head.find(attrs={"rel" : "canonical"})
        car['car_url'] = canonical['href']
        car_json = json.loads(soup.find('script', attrs={"id" : "initial-state"}).contents[0])
        car['brand'] = car_json['card']['vehicle_info']['mark_info']['code'].upper()
        
        page_card = soup.find('div', class_='PageCard')   
        card_info = page_card.find('ul', class_='CardInfo')
        car['color'] = parse_row(card_info, class_='CardInfoRow CardInfoRow_color')
        car['steering_wheel'] = car_json['card']['vehicle_info']['steering_wheel']

        car['body_type'] = car_json['card']['vehicle_info']['configuration']['body_type']
        car['description'] = car_json['card'].get('description', None)
        car['equipment_dict'] = car_json['card']['vehicle_info']['equipment']
        car['mileage'] = car_json['card']['state']['mileage']
        car['modelDate'] = car_json['card']['vehicle_info']['super_gen']['year_from']
        car['model_info'] = car_json['card']['vehicle_info']['model_info']
        car['numberOfDoors'] = car_json['card']['vehicle_info']['configuration']['doors_count']
        car['productionDate'] = car_json['card']['documents']['year']

        car['owners'] = car_json['card']['documents']['owners_number']
        car['pts_original'] = car_json['card']['documents'].get('pts_original', None)
        
        car['tech_param'] = car_json['card']['vehicle_info']['tech_param']

        car['price_info'] = car_json['card']['price_info']
        return car

In [253]:
count = 10
with open('/Volumes/Z Slim/train.json', 'w') as output:
    tree = os.walk("/Volumes/Z Slim/auto.ru")
    for folder, subfolder, filenames in tree:
        print(folder, len(filenames))
        for filename in filenames:
            if '.html' in filename:
                try:
                    page = parse_page(f'{folder}/{filename}')
                    json.dump(page, output, ensure_ascii=False)
                    output.write('\n')
                except KeyError as e:
                    print(e)
#                if count == 0:
#                    break
#                count -= 1


/Volumes/Z Slim/auto.ru 14
/Volumes/Z Slim/auto.ru/audi 2710
/Volumes/Z Slim/auto.ru/bmv 3663
/Volumes/Z Slim/auto.ru/honda 957
/Volumes/Z Slim/auto.ru/infiniti 587
/Volumes/Z Slim/auto.ru/lexus 1049
'vehicle_info'
/Volumes/Z Slim/auto.ru/mercedes 3662
/Volumes/Z Slim/auto.ru/mitsubishi 2274
/Volumes/Z Slim/auto.ru/nissan 3255
'owners_number'
/Volumes/Z Slim/auto.ru/skoda 2273
/Volumes/Z Slim/auto.ru/toyota 3554
/Volumes/Z Slim/auto.ru/volkswagen 3662
/Volumes/Z Slim/auto.ru/volvo 1213
